In [ ]:
import regex as re

In [ ]:
with open("input.txt") as input:
    data = input.read()
    lines = re.split(r":|\n\n", data)
    seeds = re.findall(r"\d+", lines[1])
    seeds = [int(x) for x in seeds]
    # Passing over one seed at a time
    for i, seed in enumerate(seeds):

        # Passing each map chunk
        for line in lines[2:]:
            range_maps = re.findall(r"\d+ \d+ \d+", line)

            # Passing over each range (line)
            for range_map in range_maps:
                destination_range_start, source_range_start, range_length = [int(x) for x in range_map.split()]

                if source_range_start <= seeds[i] and seeds[i] < source_range_start + range_length:
                    seeds[i] = destination_range_start + seeds[i] - source_range_start
                    break
            


In [ ]:
print(min(seeds))

In [ ]:
def find_range_overlap(x_start, y_start, x_end, y_end):
    return range(max(x_start, y_start), min(x_end,y_end))

with open("input.txt") as input:
    data = input.read()
    lines = re.split(r":|\n\n", data)
    seed_ranges = re.findall(r"\d+", lines[1])
    seed_ranges = [int(x) for x in seed_ranges]
    it = iter(seed_ranges)
    seed_ranges = list(zip(it,it))
    
    
    # Passing each map chunk
    for line in lines[2:]:
        range_maps = re.findall(r"\d+ \d+ \d+", line)
 
        if not range_maps:
            continue

        new_seed_ranges = []
        both_sides_overlap = range(0, 0)
        # Passing over one seed at a time
        for seed_range in seed_ranges:
            consumed = 0
            seed_range_start, seed_range_length = seed_range
            seed_range_end = seed_range_start + seed_range_length
            for range_map in range_maps:
                destination_range_start, source_range_start, range_length = [int(x) for x in range_map.split()]
                
                # Check for overlap between two ranges
                overlap = find_range_overlap(seed_range_start, source_range_start, seed_range_end, source_range_start+range_length)
                
                if overlap:
                    how_much_it_overlaps = len(overlap)
                    total_potential_overlap = min(seed_range_length, range_length)
                    how_much_is_staying_behind = total_potential_overlap - how_much_it_overlaps
                    # Entirely overlaps
                    consumed += how_much_it_overlaps
                    if how_much_it_overlaps == seed_range_length:
                        new_seed_ranges.append((destination_range_start + seed_range_start - source_range_start, how_much_it_overlaps))
                    else:
                        # if overlap is below or above
                        if seed_range_start < source_range_start and source_range_start < seed_range_end:
                            # the seed has leftovers on both sides
                            both_sides_overlap = overlap
                            new_seed_ranges.append((destination_range_start, how_much_it_overlaps))
                        elif seed_range_start < source_range_start:
                            # the seed has some leftovers on the left
                            new_seed_ranges.append((destination_range_start, how_much_it_overlaps))
                        else:
                            # the seed has some leftovers on the right
                            new_seed_ranges.append((destination_range_start + seed_range_start - source_range_start, how_much_it_overlaps))

            if consumed == 0:
                new_seed_ranges.append(seed_range)
            elif consumed < seed_range_length:
                if not find_range_overlap(seed_range_start, both_sides_overlap.start, seed_range_end, both_sides_overlap.stop):
                    new_seed_ranges.append((seed_range_start, how_much_is_staying_behind))
                else:
                    # What part of range(seed_range_start, seed_range_end) is NOT currently reflected in new_seed_range
                    new_seed_ranges.append((seed_range_start+consumed, seed_range_length-consumed))

        seed_ranges = new_seed_ranges

In [ ]:
min(seed_ranges, key = lambda t: t[0])[0]